In [30]:
import pandas as pd
df1 = pd.read_csv('df1.csv', encoding='euc-kr')
df2 = pd.read_csv('df2.csv', encoding='euc-kr' )
dfX2= pd.read_csv('dfX2.csv', encoding='euc-kr' )
df1_scale=df1.copy()

In [31]:
import numpy as np
# 로그 취하기
df1_scale['log_DayWeekend']=np.log(df1['DayWeekend'])
df1_scale['log_FemaleMaleend']=np.log(df1['FemaleMale'])
df1_scale['log_Fourteen17']=np.log(df1['Fourteen17'])
df1_scale=df1_scale[['log_DayWeekend', 'log_FemaleMaleend', 'Twenty_30', 'Six_11',
       'Eleven14',  'log_Fourteen17', 'Seventeen21', 'Twentyone24', 'AptNum',
       'AptArea', 'AptPrice', 'MonthlyIncome', 'Consume', 'ServiceSale',
       'StoreNum']]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:853: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [32]:
#상권코드 더미변수로 추가, Robust Scaling 한 후 OLS
#다중공선성 문제 발생
#R2값 상승 (0.311-> 0.445)
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

dfX=df1_scale
rb = RobustScaler()
# rb = StandardScaler()
rb.fit(dfX)
X_robust_scaled = rb.transform(dfX)
dfX2=pd.DataFrame(X_robust_scaled, columns= df1_scale.columns)
df1_robust_scaled = pd.concat([df2, dfX2], axis=1)

model = sm.OLS.from_formula("SalesPerStore  ~ C(Code) +"+"+".join(df1_robust_scaled[df1_robust_scaled.columns[3:]]), data=df1_robust_scaled)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:          SalesPerStore   R-squared:                       0.445
Model:                            OLS   Adj. R-squared:                  0.434
Method:                 Least Squares   F-statistic:                     38.13
Date:                Wed, 11 Dec 2019   Prob (F-statistic):               0.00
Time:                        09:32:50   Log-Likelihood:            -9.7324e+05
No. Observations:               49404   AIC:                         1.949e+06
Df Residuals:                   48384   BIC:                         1.957e+06
Df Model:                        1019                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           3.256e+07   1.01

In [104]:
len(df1_robust_scaled)

49415

In [119]:

result=np.zeros((2,3))

for i in range(1,len(df1_robust_scaled),5000):
    print("{},{}".format(i,i+5000))



1,5001
5001,10001
10001,15001
15001,20001
20001,25001
25001,30001
30001,35001
35001,40001
40001,45001
45001,50001


In [88]:
# KNN n_neighbors를 올려봤지만 에러가 발생했다
from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)

dfX=df1_scale
rb = RobustScaler()
# rb = StandardScaler()
rb.fit(dfX)
X_robust_scaled = rb.transform(dfX)
dfX2=pd.DataFrame(X_robust_scaled, columns= df1_scale.columns)
dfX2= dfX2.apply(lambda x: round(x,3))
df1_robust_scaled = pd.concat([df2, dfX2], axis=1)

for i in range(1,len(df1_robust_scaled),5000):
    start_num =i
    end_num =i+5000
    X= df1_robust_scaled[df1_robust_scaled.columns[4:]].loc[start_num:end_num]
    y= df1_robust_scaled['Code'].loc[start_num:end_num]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)
    nca = NeighborhoodComponentsAnalysis(random_state=42)
    knn = KNeighborsClassifier(n_neighbors=300)
    nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
    nca_pipe.fit(X_train, y_train)

    y_pred = nca_pipe.predict(X_test)

# model = sm.OLS.from_formula("SalesPerStore  ~ C(Code) +"+"+".join(df1_robust_scaled[df1_robust_scaled.columns[3:]]), data=df1_robust_scaled)

# result = model.fit()
# print(result.summary())

a=pd.DataFrame(np.array(y_test))
b=pd.DataFrame(y_pred)
KNN_result= pd.concat([a,b],axis=1)

# KNN_result.columns=['original','result']
KNN_result.columns=['original','result']
KNN_result.groupby(['original']).agg(lambda x:x.value_counts().index[0])
np.array(KNN_result)

array([[1000055, 1000073],
       [1000095, 1000073],
       [1000075, 1000073],
       ...,
       [1000064, 1000073],
       [1000060, 1000073],
       [1000079, 1000073]], dtype=int64)

In [110]:
class KNN_iter():
    def __init__(self,startnum, endnum):
        self.startnum=startnum
        self.endnum=endnum
        
    def run_KNN(self):
# KNN n_neighbors를 올려봤지만 에러가 발생했다
        from sklearn.preprocessing import RobustScaler
        # from sklearn.preprocessing import StandardScaler
        from sklearn.model_selection import train_test_split
        from sklearn.pipeline import Pipeline
        from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)

        dfX=df1_scale
        rb = RobustScaler()
        # rb = StandardScaler()
        rb.fit(dfX)
        X_robust_scaled = rb.transform(dfX)
        dfX2=pd.DataFrame(X_robust_scaled, columns= df1_scale.columns)
        dfX2= dfX2.apply(lambda x: round(x,3))
        df1_robust_scaled = pd.concat([df2, dfX2], axis=1)


        start_num =self.startnum
        end_num =self.endnum
        X= df1_robust_scaled[df1_robust_scaled.columns[4:]].loc[start_num:end_num]
        y= df1_robust_scaled['Code'].loc[start_num:end_num]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)
        nca = NeighborhoodComponentsAnalysis(random_state=42)
        knn = KNeighborsClassifier(n_neighbors=300)
        nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
        nca_pipe.fit(X_train, y_train)

        y_pred = nca_pipe.predict(X_test)

        # model = sm.OLS.from_formula("SalesPerStore  ~ C(Code) +"+"+".join(df1_robust_scaled[df1_robust_scaled.columns[3:]]), data=df1_robust_scaled)

        # result = model.fit()
        # print(result.summary())

        a=pd.DataFrame(np.array(y_test))
        b=pd.DataFrame(y_pred)
        KNN_result= pd.concat([a,b],axis=1)

        # KNN_result.columns=['original','result']
        KNN_result.columns=['original','result']
        KNN_result.groupby(['original']).agg(lambda x:x.value_counts().index[0])
        return KNN_result

In [ ]:
1,5001
5001,10001
10001,15001
15001,20001
20001,25001
25001,30001
30001,35001
35001,40001
40001,45001
45001,50001

In [121]:
result1=KNN_iter(1,5001)
result1.run_KNN()
result2=KNN_iter(5001,10001)
result2.run_KNN()
result3=KNN_iter(10001,15001)
result3.run_KNN()



,original,result
0,1000961,1000974
1,1000034,1000896
2,1000042,1000073
3,1000936,1000032
4,1000913,1000177
...,...,...
2996,1000128,1000220
2997,1000882,1000866
2998,1000193,1000896
2999,1000931,1000974


In [125]:
df = pd.concat([result1.run_KNN(),result2.run_KNN()], axis=0)

In [127]:
result3=KNN_iter(10001,15001)
df = pd.concat([df,result3.run_KNN()], axis=0)

In [131]:
df.to_csv("result1.csv", sep=",", encoding="euc-kr", index=False)

In [135]:
df.tail()

,original,result
2996,1000128,1000220
2997,1000882,1000866
2998,1000193,1000896
2999,1000931,1000974
3000,1000199,1000073


_ result4부터는 에러가 뜬다
- 아무래도 변수가 많아서 ram 이 감당하지 못하는것 같다
- 변수 삭제 방법은 다음과 같다
```
%whos: 변수명 검색
!del 변수: 변수삭제
%reset: 모든 변수 삭제
```

- 하지만 우선 적은 데이터로 한번 끝까지 돌려보는게 필요해서 15000개의 데이터만 가지고 결과를 출력해보도록 하겠다

In [142]:
df_grouped=df.groupby(['original']).agg(lambda x:x.value_counts().index[0])
df_grouped

,result
original,
1000001,1000073
1000002,1000073
1000003,1000073
1000004,1000073
1000005,1000032
...,...
1001006,1000896
1001007,1000073
1001008,1000119


In [143]:
len(df['result'].unique())

47

### 소결론
- 메모리 한계로 인해 0행~15000행까지의 데이터만 가지고  KNN 클러스터링을 실시했다.
- n_neighbors=300 으로 하니깐 1004개의 지역코드가 47개로 군집화 되었다.

### 앞으로 할 내용
- 지역코드 47개로 군집화 한 것이 다중공선성을 피할 수 있는 정도인지 OLS분석이 필요하다
- 지역코드가 어떻게 군집화 되었는지 지도로 표시해야한다(하지만 어떤 library를 사용할지, 어떻게 47개를 구분해서 나타낼지는 좀더 조사해야한다)
- 만약 성공하면 나머지 (49415-15000)개의 데이터를 KNN학습시켜야 한다

In [ ]:
#나중에 나머지 데이터 KNN실시하기위한 코드를 메모해놓도록 하겠다

In [134]:
# result4=KNN_iter(15001,20000)
# df = pd.concat([df,result4.run_KNN()], axis=0)

```
result5=KNN_iter(20001,25001)
result5.run_KNN()
result6=KNN_iter(25001,30001)
result6.run_KNN()
result7=KNN_iter(30001,35001)
result7.run_KNN()
result8=KNN_iter(35001,40001)
result8.run_KNN()
result9=KNN_iter(40001,45001)
result9.run_KNN()
result10=KNN_iter(45001,49415)
result10.run_KNN()
```

In [80]:
# np.unique(y_pred)

In [79]:
# np.unique(np.array(y_test))